In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
torch.set_default_dtype(torch.float64)
import numdifftools as nd

In [3]:
def generateData(size = 1000):
    def mse(x,y):
        x.requires_grad_(True)
        y.requires_grad_(True)
        mse = torch.sum((torch.abs(x - y))**2)
        mse.requires_grad_(True)
        return mse
    reference = torch.FloatTensor([1e-9, 1e-9, 1e-9])
    xy = torch.FloatTensor(size, 2).uniform_(-2.1, 2.1)
    z = torch.FloatTensor(size, 1).uniform_(0., 1)
    xtrain = torch.cat((xy, z), dim=-1) #VSTACK
    ytrain = torch.stack([mse(x, reference) for x in xtrain], 0).reshape(size, 1) ## Append to torch tensor

    return xtrain, ytrain
xTrain, yTrain = generateData(size = 1000)
xTest, yTest = generateData(1000)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import torch.utils.data as data_utils
torch.set_default_dtype(torch.float64)


class SquaredNet(nn.Module):
    def __init__(self, input_features, output_features):
        super().__init__()
        
        self.input_features = input_features
        self.output_features = output_features

        self.fc1 = nn.Linear(self.input_features, 8)
        f1 = 1 / np.sqrt(self.fc1.weight.data.size()[0])
        torch.nn.init.normal_(self.fc1.weight.data, -f1, f1)
        torch.nn.init.uniform_(self.fc1.bias, -f1, f1)        
        

        self.fc2 = nn.Linear(8, 8)
        f2 = 1 / np.sqrt(self.fc2.weight.data.size()[0])
        torch.nn.init.normal_(self.fc2.weight.data, -f2, f2)
        torch.nn.init.uniform_(self.fc2.bias, -f2, f2)        
        
                
        self.fc3 = nn.Linear(8, 3)
        f3 = 1 / np.sqrt(self.fc3.weight.data.size()[0])
        torch.nn.init.normal_(self.fc3.weight.data, -f3, f3)
        torch.nn.init.uniform_(self.fc3.bias, -f3, f3)   
        
        self.fc4 = nn.Linear(3, self.output_features)        
        self.device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        
        x = torch.tanh(self.fc1(x)) 

        x = torch.tanh(self.fc2(x))

        x = torch.tanh(self.fc3(x))
        
        x4 = self.fc4(x)**2
        print(x4)
        return x4
    
    
def mse_loss(x, y):
    return torch.sum((torch.abs(x - y))**2)
    
def trainNetwork(net, start, cost, epochs = 3):
    from tqdm import tqdm
    
    net = net.float()
    net.train()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    n_epochs = epochs
    criterion = nn.CrossEntropyLoss()
    for epoch in tqdm(range(n_epochs)):
        for x, y in zip(start, cost):
            x.to('cpu')
            y.to('cpu')
            x.requires_grad_=True
            y_ = net(x)
            
            
            loss = F.mse_loss(y_, y)
            optimizer.zero_grad()
            
            loss.backward(retain_graph=True)
            optimizer.step()  
    del start, cost        
    return net
    

In [18]:
n = SquaredNet(3, 1)

In [19]:
x = np.random.rand(1, 3)


In [20]:
def get_jacobian(net, x, noutputs):
    x = x.squeeze()
    n = x.size()[0]
    x = x.repeat(noutputs, 1)
    x.requires_grad_(True)
    y = net(x)
    y.backward(torch.eye(noutputs))
    return x.grad.data                                           


In [21]:
a1 = torch.as_tensor(x, dtype = torch.float64)


In [22]:
n(a1)

tensor([[0.0015]], grad_fn=<PowBackward0>)


tensor([[0.0015]], grad_fn=<PowBackward0>)

In [23]:
get_jacobian(n, a1, 1)

tensor([[0.0015]], grad_fn=<PowBackward0>)


tensor([[-0.0004, -0.0004, -0.0003]])